<a href="https://colab.research.google.com/github/niikun/introduction_LLM/blob/main/rag_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/drive/MyDrive/coursera/MLOps/Introduction/top_rated_wines.csv")
df.head()

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."


In [3]:
df.shape

(1365, 5)

In [4]:
df = df.sample(102,random_state=0)
df[df.isnull().sum(axis=1)>0]

,name,region,variety,rating,notes
45,Alvear Pedro Ximenez 1830 Solera (500ML),Spain,NaN,97.0,The 1830 Solera is black with shades of chocol...
924,Dominus Estate 2017 6-Pack Wood Case (OWC),"Napa Valley, California",NaN,97.0,"The historic Napanook Vineyard, a 124-acre sit..."


In [5]:
df.dropna(inplace=True)
df.shape

(100, 5)

In [6]:
data = df.to_dict("records")
data[:3]

[{'name': 'G.D. Vajra Langhe Freisa Kye 2008',
  'region': 'Piedmont, Italy',
  'variety': 'Red Wine',
  'rating': 96.0,
  'notes': 'Freisa is an ancient and noble variety of grape. A relative of Nebbiolo, it shares the same character, crispy acidity and tannic structure.  Freisa saves the same taste the variety had centuries ago in its own genes. Made with 100% Freisa grapes.'},
 {'name': 'Flowers Sea View Ridge Pinot Noir 2010',
  'region': 'Sonoma County, California',
  'variety': 'Red Wine',
  'rating': 98.0,
  'notes': "Deep vibrant red color. Delicate floral notes of rose petal and bergamot combine with hints of forest floor, earth and ripe cherry. Flavors emerge with raspberry, red current and dried cherry with hints of graphite and other minerals. The wine's texture is elegant and smooth across the palate with an enduring finish."},
 {'name': 'Larkmead Solari Cabernet Sauvignon (bin soiled label) 2006',
  'region': 'Napa Valley, California',
  'variety': 'Red Wine',
  'rating':

In [7]:
!pip install qdrant-client

In [8]:
!pip install -U sentence-transformers

In [9]:
!pip install --upgrade transformers protobuf

In [11]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [13]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
qdrant = QdrantClient(":memory:")

In [15]:
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

<ipython-input-15-19084fd88045>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [17]:
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx,doc in enumerate(data)
    ]
)

In [35]:
hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode("a Malbec wine from Argentina").tolist(),
    limit=3
)
search_results = [hit.payload for hit in hits]
search_results

[{'name': 'La Vizcaina La Vitoriana Tinto 2018',
  'region': 'Bierzo, Spain',
  'variety': 'Red Wine',
  'rating': 97.0,
  'notes': 'La Vizcaina La Vitoriana offers aromas of reinette apple, ripe blackberries, light touches of pilonga chestnut, and a background of autumn earth, with points of black licorice and a balsamic finish. On the palate it is a very penetrating wine, structured, with subtle nuances of wood and a sweet and appetising finish. It is a wine with a long finish.'},
 {'name': 'Caymus Special Selection Cabernet Sauvignon 1987',
  'region': 'California',
  'variety': 'Red Wine',
  'rating': 98.0,
  'notes': 'A superlative wine from a great vintage. Aging beautifully with years to go. Very limited.'},
 {'name': 'Booker Vineyard Jada 2015',
  'region': 'Paso Robles, Central Coast, California',
  'variety': 'Red Wine',
  'rating': 96.0,
  'notes': 'This Grenache heavy wine is very elegant and pretty, but in no way a light and simple wine. It is red fruits galore, mostly ras

In [23]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 6.5 MB/s eta 0:00:00


In [31]:
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = api_key

In [36]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining about wines in Japanese."},
    {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
    {"role":"assistant","content":str(search_results)}
  ]
)

print(completion.choices[0].message.content)

こちらの素晴らしいマルベックワインをお勧めします。

**ワイン名:** 
ラ・ヴィスカイナ・ラ・ビトリアーナ・ティント 2018

**地域:**
アルゼンチン

**特徴:**
ラ・ヴィスカイナ・ラ・ビトリアーナは、リネットリンゴの香り、熟れたブラックベリー、ピロンガ栗の軽いタッチ、秋の大地の背景、黒いリコリスのポイント、バルサミックなフィニッシュが特徴です。口に含むと、非常に浸透力のあるワインで、構造がしっかりしており、微妙なウッディヌアンスと甘く食欲をそそるフィニッシュがあります。長い余韻が楽しめる素晴らしいワインです。
